In [2]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


env = gym.make("CartPole-v1")
env.reset()
goal_steps = 500
score_requirement = 60
initial_games = 100000

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [9]:
def jogos_aleatórios_primeiro():
    #isso é apenas uma visualização de jogos com ações aleatórias, não é necessário.
    for episode in range(5):
        env.reset()
        for t in range(200):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break
                
jogos_aleatórios_primeiro()

In [10]:
def população_inicial():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            observation, reward, done, info = env.step(action)
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break
        #aqui nós salvamos os jogos com pontos acima de um certo valor
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                #convertemos para one-hot
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                training_data.append([data[0], output])
        env.reset()
        scores.append(score)
        
    print('Score médio aceitado:',mean(accepted_scores))
    print('Mediana dos scores aceitados:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [11]:
def construir_modelo(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())

    return model

def treinar_modelo(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    print(X)
    print(y)
    model = construir_modelo(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model

In [12]:
training_data = população_inicial()

Score médio aceitado: 71.66470971950424
Mediana dos scores aceitados: 68.0
Counter({60.0: 130, 61.0: 113, 63.0: 113, 62.0: 106, 67.0: 75, 65.0: 71, 66.0: 71, 64.0: 64, 71.0: 60, 68.0: 58, 69.0: 55, 70.0: 54, 72.0: 47, 73.0: 40, 75.0: 37, 77.0: 34, 76.0: 33, 74.0: 30, 81.0: 28, 82.0: 26, 79.0: 22, 78.0: 22, 84.0: 19, 80.0: 18, 85.0: 18, 86.0: 16, 87.0: 13, 83.0: 13, 96.0: 12, 88.0: 12, 91.0: 11, 100.0: 9, 89.0: 7, 102.0: 7, 94.0: 6, 101.0: 6, 90.0: 6, 99.0: 5, 93.0: 5, 95.0: 4, 92.0: 4, 115.0: 4, 106.0: 4, 114.0: 3, 119.0: 3, 104.0: 3, 108.0: 3, 103.0: 3, 98.0: 3, 113.0: 3, 97.0: 3, 107.0: 3, 125.0: 2, 105.0: 2, 109.0: 2, 144.0: 1, 131.0: 1, 156.0: 1, 112.0: 1, 124.0: 1, 137.0: 1, 110.0: 1, 111.0: 1, 118.0: 1, 117.0: 1, 133.0: 1, 138.0: 1})


In [13]:
trained_model = treinar_modelo(training_data)

[[-0.02000112  0.21273008  0.03683412 -0.29778864]
 [-0.01574651  0.40730813  0.03087835 -0.57863096]
 [-0.00760035  0.21176735  0.01930573 -0.27638281]
 ...
 [-0.19299207 -0.36904536 -0.18474997 -0.52268889]
 [-0.20037298 -0.56115221 -0.19520375 -0.29344048]
 [-0.21159602 -0.36386075 -0.20107256 -0.64078342]]
[[0 1]
 [1 0]
 [1 0]
 ...
 [1 0]
 [0 1]
 [1 0]]
Train on 108329 samples
Epoch 1/10
108329/108329 [==============================] - 3s 27us/sample - loss: 0.2346
Epoch 2/10
108329/108329 [==============================] - 5s 43us/sample - loss: 0.2315
Epoch 3/10
108329/108329 [==============================] - 4s 38us/sample - loss: 0.2310
Epoch 4/10
108329/108329 [==============================] - 4s 38us/sample - loss: 0.2305
Epoch 5/10
108329/108329 [==============================] - 4s 35us/sample - loss: 0.2302
Epoch 6/10
108329/108329 [==============================] - 4s 33us/sample - loss: 0.2300
Epoch 7/10
108329/108329 [==============================] - 4s 33us/sample -

In [14]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Score médio:',sum(scores)/len(scores))
print('Direita:{}  Esquerda:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

[500.0, 102.0, 500.0, 500.0, 162.0, 134.0, 176.0, 500.0, 104.0, 500.0]
Score médio: 317.8
Direita:0.4977973568281938  Esquerda:0.5022026431718062
